In [1]:
import string
import numpy as np
import pandas as pd
from model import *
from prompt import *
from batch import *
from generate_json import *

from tqdm import tqdm
tqdm.pandas()

In [2]:
data1 = pd.read_csv('works-2018-2022.csv', low_memory=False)
data2 = pd.read_csv('works-2023-2024.csv', low_memory=False)
df = pd.concat([data1, data2])
print(len(df))
df['publication_date'] = pd.to_datetime(df['publication_date'])
df['created_date'] = pd.to_datetime(df['created_date'])
df = df[df['created_date'] >= '2018-10-11']
df = df.sort_values(by='created_date')
print(len(df))
df.head(5)

100400
97242


,id,doi,title,display_name,relevance_score,publication_year,publication_date,language,type,type_crossref,...,locations.source.type,locations.source,sustainable_development_goals.id,sustainable_development_goals.score,sustainable_development_goals.display_name,grants.funder,grants.funder_display_name,grants.award_id,counts_by_year.year,counts_by_year.cited_by_count
31287,https://openalex.org/W2894846460,NaN,Zooming Network,Zooming Network,2.071526,2018,2018-10-04,en,preprint,posted-content,...,repository,NaN,https://metadata.un.org/sdg/16,0.75,"Peace, justice, and strong institutions",NaN,NaN,NaN,2019,1
5324,https://openalex.org/W2895320328,https://doi.org/10.1039/c8na00090e,Porous tal palm carbon nanosheets: preparation...,Porous tal palm carbon nanosheets: preparation...,10.670827,2019,2019-01-01,en,article,journal-article,...,journal|repository|repository,nan|nan|nan,NaN,NaN,NaN,https://openalex.org/F4320322323|https://opena...,King Fahd University of Petroleum and Minerals...,None|None,2024|2023|2022|2021|2020|2019,8|23|15|21|10|8
46082,https://openalex.org/W2895487477,https://doi.org/10.1063/1.5062649,Overall assessment on flexible pavement mainte...,Overall assessment on flexible pavement mainte...,0.540709,2018,2018-01-01,en,article,proceedings-article,...,journal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10532,https://openalex.org/W2895623723,https://doi.org/10.1016/j.renene.2018.10.002,Potential use of flocculating oleaginous yeast...,Potential use of flocculating oleaginous yeast...,5.783283,2019,2019-06-01,en,article,journal-article,...,journal,NaN,https://metadata.un.org/sdg/12,0.43,Responsible consumption and production,https://openalex.org/F4320322614,Thailand Research Fund,RTA 6080010,2023|2022|2021|2020|2019,4|5|6|9|1
20957,https://openalex.org/W2894642382,https://doi.org/10.1038/s41598-018-33402-1,Attenuation of hypertension by C-fiber stimula...,Attenuation of hypertension by C-fiber stimula...,3.329368,2018,2018-10-08,en,article,journal-article,...,journal|repository|repository|repository|journ...,nan|nan|nan|nan|nan|nan|nan|nan,https://metadata.un.org/sdg/3|https://metadata...,0.82|0.82,Good health and well-being|Good health and wel...,https://openalex.org/F4320322113|https://opena...,Korea Institute of Oriental Medicine|National ...,K18181|2017R1E1A2A01079599|K18181|2017R1E1A2A0...,2024|2023|2022|2020|2019|2024|2023|2022|2020|2019,1|5|3|2|1|1|5|3|2|1


In [ ]:
# import matplotlib.pyplot as plt

# df['created_date'] = pd.to_datetime(df['created_date'])
# df.set_index('created_date', inplace=True)

# # Resample by month and count papers
# monthly_papers = df.resample('M').size()

# plt.figure(figsize=(12, 6))
# monthly_papers.plot(kind='line', marker='o')
# plt.title('Monthly Change in Number of Papers')
# plt.xlabel('Date')
# plt.ylabel('Number of Papers')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [3]:
for i in df.columns:
    print(i)

id
doi
title
display_name
relevance_score
publication_year
publication_date
language
type
type_crossref
indexed_in
countries_distinct_count
institutions_distinct_count
corresponding_author_ids
corresponding_institution_ids
apc_list
apc_paid
fwci
has_fulltext
cited_by_count
is_retracted
is_paratext
locations_count
datasets
versions
referenced_works_count
referenced_works
related_works
ngrams_url
cited_by_api_url
updated_date
created_date
ids.openalex
ids.doi
ids.mag
primary_location.is_oa
primary_location.landing_page_url
primary_location.pdf_url
primary_location.source.id
primary_location.source.display_name
primary_location.source.issn_l
primary_location.source.issn
primary_location.source.is_oa
primary_location.source.is_in_doaj
primary_location.source.host_organization
primary_location.source.host_organization_name
primary_location.source.host_organization_lineage
primary_location.source.host_organization_lineage_names
primary_location.source.type
primary_location.license
primary_lo

### 1. Perform initial cleaning

This part aims to initially address the collected data. 

- Only consider preprint and article for analysis
- Remove duplicates

- Whether to remove records without doi?

In [4]:
types = ['preprint', 'article']

df1 = df[df['type'].isin(types)]
print(len(df1))

87489


In [5]:
def preprocess_text(text):
    """Remove punctuation and lowercase the title."""
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower().strip()
    return text

def remove_duplicates(df):
    """Remove duplicates and keep the last one."""
    df['title'] = df['title'].astype(str)
    df['processed_title'] = df['title'].apply(preprocess_text)
    df = df.drop_duplicates(subset=['processed_title'], keep='last')
    df = df.drop(columns=['processed_title'])
    return df

df2 = remove_duplicates(df1)
print(len(df2))

/var/folders/cw/l2jwfcqs4yl3k4j6btbtvjtm0000gr/T/ipykernel_34043/3581942531.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].astype(str)
/var/folders/cw/l2jwfcqs4yl3k4j6btbtvjtm0000gr/T/ipykernel_34043/3581942531.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['processed_title'] = df['title'].apply(preprocess_text)
/var/folders/cw/l2jwfcqs4yl3k4j6btbtvjtm0000gr/T/ipykernel_34043/3581942531.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

70644


### 2. Address potential duplicates

This part aims to address potential duplicates by calculating the similarity between Preprints and Articles

- Paper title
- Authors

After reviewing all the preprints with similarity score higher than 0.5, we found the threshold of 0.6 is a good one for the filtering process. 

In [ ]:
def jaccard_similarity(str1, str2):
    """Compute Jaccard similarity between two strings."""
    str1 = preprocess_text(str1)
    str2 = preprocess_text(str2)
    set1 = set(str1.split())
    set2 = set(str2.split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0


def find_potential_article(preprint_row, articles_df, threshold=0.5):
    """Find a potential published article for a given preprint."""
    preprint_date = preprint_row['publication_date']
    preprint_topic = preprint_row['primary_topic.field.display_name']
    preprint_title_author = preprint_row['title_author']
    
    # Filter articles by date and topic
    potential_articles = articles_df[
        (articles_df['publication_date'] > preprint_date) &
        (articles_df['primary_topic.field.display_name'] == preprint_topic)
    ]
    if potential_articles.empty:
        return pd.Series({'matched_article_title_author': None, 'matched_article_date': None, 'similarity': None})
    
    # Compute similarities and fine the best match
    similarities = []
    for _, article in potential_articles.iterrows():
        sim = jaccard_similarity(preprint_title_author, article['title_author'])
        similarities.append((sim, article['title_author'], article['publication_date']))
    if similarities:
        best_match = max(similarities, key=lambda x: x[0])
        if best_match[0] >= threshold:
            return pd.Series({
                'matched_article_title_author': best_match[1],
                'matched_article_date': best_match[2],
                'similarity': best_match[0]
            })
    return pd.Series({'matched_article_title_author': None, 'matched_article_date': None, 'similarity': None})


def remove_similar_duplicates(df):
    """Remove duplicates based on similarity score from a DataFrame."""
    df['authors'] = df['authorships.raw_author_name'].fillna('').astype(str).str.split('|').apply(lambda x: ', '.join(x) if x else '')
    df['title_author'] = df['title'] + ' - ' + df['authors']
    dff = df[['doi', 'type', 'title_author', 'publication_date', 'primary_topic.field.display_name']]
    dff = dff.sort_values(by='publication_date', ascending=True)
    preprints = dff[dff['type'] == 'preprint']
    articles = dff[dff['type'] == 'article']
    # Apply the function to each preprint
    results = []
    for _, preprint in tqdm(preprints.iterrows(), total=len(preprints), desc="Processing preprints"):
        result = find_potential_article(preprint, articles)
        results.append(result)
    # Combine results with preprints
    preprints_with_matches = pd.concat([preprints.reset_index(drop=True), pd.DataFrame(results)], axis=1)
    matched_preprints = preprints_with_matches.dropna(subset=['matched_article_title_author'])
    return matched_preprints

matched_preprints = remove_similar_duplicates(df2)
matched_preprints.to_csv('preprint_article_matches.csv', index=False)

In [6]:
matched_preprints = pd.read_csv('preprint_article_matches.csv')

In [7]:
import math

remove_preprints = matched_preprints[matched_preprints['similarity'] >= 0.6]
remove_list = remove_preprints['doi'].tolist()
remove_list = [x for x in remove_list if not (isinstance(x, float) and math.isnan(x))]

df3 = df2[~df2['doi'].isin(remove_list)]
print(len(df3))

69717


### Check related papers

This section aims to verify if a paper aligns with our study scope, as some papers may include the search term but not discuss LLM applications. We check papers in two steps. 

- Step 1: If the major topic of this paper focuses on natural langauge processing (NLP), artificial intelligence (AI), or any NLP-related tasks, we consider the paper's topic is related.

- Step 2: For those papers with the topic not related to any NLP tasks, we use the GPT-4o model to analyze the title and abstract (if available) while also considering the primary topics addressed in the paper.

In [8]:
related_topics = ['natural langauge processing', 'language processing', 'artificial intelligence', 'machine learning', 
                  'deep learning', 'transfer learning', 'data mining', 'text mining', 'speech recognition', 'multimodal',
                  'question answering', 'text analysis', 'topic modeling', 'translation', 'sentiment analysis', 
                  'named entity recognition', 'language analysis']

def check_related(topics, keywords):
    topics = topics.lower().strip()
    keywords = keywords.lower().strip()
    if any(rt in topics for rt in related_topics) or any(rt in keywords for rt in related_topics):
        return 'Yes'
    else:
        return 'No'

df3['topics.display_name'] = df3['topics.display_name'].astype(str)
df3['keywords.display_name'] = df3['keywords.display_name'].astype(str)
df3['possible'] = df3.apply(lambda x: check_related(x['topics.display_name'], x['keywords.display_name']), axis=1)
df4 = df3[df3['possible'] == 'Yes']
df5 = df3[df3['possible'] == 'No'] # These papers are checked by GPT4o
print(len(df4), len(df5))

/var/folders/cw/l2jwfcqs4yl3k4j6btbtvjtm0000gr/T/ipykernel_34043/337725868.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['topics.display_name'] = df3['topics.display_name'].astype(str)
/var/folders/cw/l2jwfcqs4yl3k4j6btbtvjtm0000gr/T/ipykernel_34043/337725868.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['keywords.display_name'] = df3['keywords.display_name'].astype(str)


42454 27263


/var/folders/cw/l2jwfcqs4yl3k4j6btbtvjtm0000gr/T/ipykernel_34043/337725868.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df3['possible'] = df3.apply(lambda x: check_related(x['topics.display_name'], x['keywords.display_name']), axis=1)
/var/folders/cw/l2jwfcqs4yl3k4j6btbtvjtm0000gr/T/ipykernel_34043/337725868.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['possible'] = df3.apply(lambda x: check_related(x['topics.display_name'], x['keywords.display_name']), axis=1)


In [9]:
df5 = pd.read_csv('check_papers_response.csv')
df6 = pd.concat([df4, df5])
df6 = df6.drop_duplicates(subset=['title'], keep='last')
print(len(df5), len(df6))

/var/folders/cw/l2jwfcqs4yl3k4j6btbtvjtm0000gr/T/ipykernel_34043/277603897.py:1: DtypeWarning: Columns (23,24,35,52,53,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  df5 = pd.read_csv('check_papers_response.csv')


27263 69717


In [10]:
df6['possible'] = df6['possible'].str.replace('Yes.', 'Yes').replace('No.', 'No')
df7 = df6[df6['possible'] == 'Yes']
print(len(df7))
#df7.to_csv('cleaned_papers.csv', index=False)

50391


### Relevance validation

In [12]:
from sklearn.metrics import classification_report

check_df = pd.read_excel('check_papers_samples.xlsx')
check_df = check_df.dropna(subset=['Human-annotation'])
possible = check_df['possible']
human_annotation = check_df['Human-annotation']

possible_binary = possible.map({'Yes': 1, 'No': 0})
human_annotation_binary = human_annotation.map({'Yes': 1, 'No': 0})

report = classification_report(human_annotation_binary, possible_binary)
print(report)

              precision    recall  f1-score   support

           0       0.91      1.00      0.96        86
           1       1.00      0.92      0.96       104

    accuracy                           0.96       190
   macro avg       0.96      0.96      0.96       190
weighted avg       0.96      0.96      0.96       190

